In [ ]:
import pandas as pd
import spacy

df_train= pd.read_csv("politicES_phase_2_train_public.csv", on_bad_lines='skip')
df_test= pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')

In [ ]:
# Agrupar los tweets por 'label' y 'ideology_multiclass' y luego concatenar los tweets
df_train= df_train.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()
df_test= df_test.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()

In [ ]:
df_train.drop('label', axis=1, inplace=True)
df_test.drop('label', axis=1, inplace=True)

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load('es_core_news_lg')

def preprocess_text(tweet):
    # Procesa el tweet usando el modelo de spaCy
    doc = nlp(tweet)
    # Lematiza y elimina stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [ ]:
df_train['clean_tweet'] = df_train['tweet'].apply(preprocess_text)
df_test['clean_tweet'] = df_test['tweet'].apply(preprocess_text)

In [ ]:
df_train.to_csv('train_tokenized.csv')
df_test.to_csv('test_tokenized.csv')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df_train= pd.read_csv('/content/drive/My Drive/TFM/train_tokenized.csv')
df_test= pd.read_csv('/content/drive/My Drive/TFM/test_tokenized.csv')

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_train['ideology_multiclass'] = label_encoder.fit_transform(df_train['ideology_multiclass'])
df_test['ideology_multiclass'] = label_encoder.fit_transform(df_test['ideology_multiclass'])

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import LongformerModel, LongformerTokenizer

tokenizer = LongformerTokenizer.from_pretrained('PlanTL-GOB-ES/longformer-base-4096-bne-es')
model = LongformerModel.from_pretrained('PlanTL-GOB-ES/longformer-base-4096-bne-es')
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

...

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'LongformerTokenizer'.


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of LongformerModel were not initialized from the model checkpoint at PlanTL-GOB-ES/longformer-base-4096-bne-es and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LongformerModel(
  (embeddings): LongformerEmbeddings(
    (word_embeddings): Embedding(50262, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
  )
  (encoder): LongformerEncoder(
    (layer): ModuleList(
      (0-11): 12 x LongformerLayer(
        (attention): LongformerAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
    

In [6]:
import numpy as np

encodings = tokenizer(df_train['clean_tweet'].tolist(), return_tensors="pt", padding='max_length', max_length=4096, truncation=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Ajuste el tamaño del lote
batch_size = 2  # Ajusta según la capacidad de tu GPU

# Función para procesar los textos en lotes
def batch_encode(model, inputs, batch_size=2):
    batch_start = 0
    batch_end = batch_size
    all_embeddings = []

    while batch_start < len(inputs['input_ids']):
        batch_inputs = {k: v[batch_start:batch_end] for k, v in inputs.items()}
        batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()}
        with torch.no_grad():
            outputs = model(**batch_inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1)
            all_embeddings.append(embeddings.cpu().numpy())
        batch_start += batch_size
        batch_end += batch_size

    return np.vstack(all_embeddings)

# Llama a la función
embeddings = batch_encode(model, encodings, batch_size)


In [7]:

y = df_train['ideology_multiclass']

# Dividir el conjunto de entrenamiento para validación (10% para test)
X_train, X_val, y_train, y_val = train_test_split(embeddings, y, test_size=0.2, random_state=42)

# Definir varios modelos para probar
modelos = {
    'XGB': xgb.XGBClassifier(),
    'Regresión Logística': LogisticRegression(),
    'SVM': SVC(),

}

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_val)
    macro_f1 = f1_score(y_val, y_pred, average='macro')
    print(f"{nombre} - Macro F1 Score: {macro_f1}")

XGB - Macro F1 Score: 0.840299005382345


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Regresión Logística - Macro F1 Score: 0.6002750652787536
SVM - Macro F1 Score: 0.17310874704491724


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score


# Definir el modelo y los parámetros para GridSearchCV
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Definir los parámetros para el Grid Search
param_grid = {
    'max_depth': [3, 5, 7, None],  # Profundidades del árbol
    'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'n_estimators': [100, 200],  # Número de árboles
    'subsample': [0.8, 1]  # Porcentaje de muestras usadas por árbol
}
# Crear un GridSearchCV
clf = GridSearchCV(model, param_grid, scoring='f1', cv=5, verbose=1, n_jobs=-1)

# Ajustar el modelo
clf.fit(X_train, y_train)

# Mejores parámetros y mejor score
print("Mejores parámetros:", clf.best_params_)
print("Mejor puntuación de cross-validation (macro F1):", clf.best_score_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Mejores parámetros: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Mejor puntuación de cross-validation (macro F1): nan


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score

# Definir el modelo y los parámetros para GridSearchCV
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [10]:
tokenizer = LongformerTokenizer.from_pretrained('PlanTL-GOB-ES/longformer-base-4096-bne-es')
model2 = LongformerModel.from_pretrained('PlanTL-GOB-ES/longformer-base-4096-bne-es')
model2.eval()

encodings_test = tokenizer(df_test['clean_tweet'].tolist(), return_tensors="pt", padding='max_length', max_length=4096, truncation=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model2 = model2.to(device)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'LongformerTokenizer'.
Some weights of LongformerModel were not initialized from the model checkpoint at PlanTL-GOB-ES/longformer-base-4096-bne-es and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
X_test = batch_encode(model2, encodings_test, batch_size)
y_test = df_test['ideology_multiclass']

In [12]:
y_pred = model.predict(X_test)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Macro F1 en el conjunto de prueba:", macro_f1)
print(classification_report(y_test, y_pred))

Macro F1 en el conjunto de prueba: 0.4737400947724839
              precision    recall  f1-score   support

           0       0.47      0.38      0.42       117
           1       0.57      0.79      0.66       210
           2       0.52      0.47      0.49       153
           3       0.64      0.21      0.31        67

    accuracy                           0.54       547
   macro avg       0.55      0.46      0.47       547
weighted avg       0.54      0.54      0.52       547

